In [8]:
def conj_set_preimage(self, other, num_cpus = 1):
    from itertools import product
    from copy import deepcopy
    from sage.parallel.use_fork import p_iter_fork
    f = copy(self)
    g = copy(other)
    n = f.domain().dimension_relative()
    if (n == 1) and (R in NumberFields() or R in FiniteFields())\
        and (f.sigma_invariants(1) != g.sigma_invariants(1)):
        return []
    
    def isLinearlyIndependent(S):
        M = matrix([list(t) for t in S])
        if len(S) < n+1:
            if M.rank() == len(S):
                return True
            return False
        else:
            if any(l == 0 for l in M.minors(n+1)):
                return False
            return True

    L = Set(f.periodic_points(1))
    K = Set(g.periodic_points(1))
    if len(L) != len(K):  # checks maps have the same number of fixed points
        return []
    mult_to_point_L = {}
    mult_to_point_K = {}
    point_to_mult_L = {}
    point_to_mult_K = {}
    level = 0
    for i in range(len(L)):
        mult_L = f.multiplier(L[i], 1).charpoly()
        mult_K = g.multiplier(K[i], 1).charpoly()
        tup_L = (mult_L, level)
        tup_K = (mult_K, level)
        if tup_L not in mult_to_point_L:
            mult_to_point_L[tup_L] = [L[i]]
        else:
            mult_to_point_L[tup_L] += [L[i]]
        if tup_K not in mult_to_point_K:
            mult_to_point_K[tup_K] = [K[i]]
        else:
            mult_to_point_K[tup_K] += [K[i]]
        point_to_mult_L[L[i]] = (mult_L, level)
        point_to_mult_K[K[i]] = (mult_K, level)
    #print('mult_to_point_L:', mult_to_point_L)
    #print('mult_to_point_K:', mult_to_point_K)
    #print('point_to_mult_L:', point_to_mult_L)
    #print('point_to_mult_K:', point_to_mult_K)
    repeated_mult_L = {}
    for mult_L in mult_to_point_L:
        repeated = len(mult_to_point_L[mult_L])
        if mult_L not in mult_to_point_K:
            return []
        elif len(mult_to_point_K[mult_L]) != repeated:
            return []
        if repeated not in repeated_mult_L:
            repeated_mult_L[repeated] = [mult_to_point_L[mult_L]]
        else:
            repeated_mult_L[repeated] += [mult_to_point_L[mult_L]]
    #print('repeated_mult:', repeated_mult_L)
    d = len(L)
    r = f.domain().base_ring()
    more = True
    source = []
    corresponding = []
    for r in sorted(repeated_mult_L.keys()):
        for point_lst in repeated_mult_L[r]:
            for point in point_lst:
                if isLinearlyIndependent(source + [point]):
                    source.append(point)
                    mult = point_to_mult_L[point]
                    # if another point with this multiplier and level pair is in S
                    # then the multiplier level pair will be the last element of corresponding
                    if len(corresponding) != 0:
                        if corresponding[-1][0] == mult:
                            corresponding[-1][1] += 1
                        else:
                            corresponding.append([mult, 1])
                    else:
                        corresponding.append([mult, 1])
                if len(source) == n+2:
                    more = False
                    break
            if len(source) == n+2:
                break
        if len(source) == n+2:
            break
    #print('corresponding:', corresponding)
    i_repeated_mult = deepcopy(repeated_mult_L)
    a_repeated_mult = {}
    found_no_more = True
    while more:
        level += 1
        for r in i_repeated_mult:
            for point_lst_L in i_repeated_mult[r]:
                old_tup_L = point_to_mult_L[point_lst_L[0]]
                #print('current tup:', old_tup_L)
                point_lst_K = mult_to_point_K[old_tup_L]
                mult_L = old_tup_L[0]
                Tl = []
                Tk = []
                for pnt in point_lst_L:
                    for preimage in f.rational_preimages(pnt):
                        if preimage != pnt:
                            Tl.append(preimage)
                for pnt in point_lst_K:
                    for preimage in g.rational_preimages(pnt):
                        if preimage != pnt:
                            Tk.append(preimage)
                #print('Tl:', Tl)
                #print('Tk:', Tk)
                if len(Tl) != len(Tk):
                    return []
                if len(Tl) != 0:
                    found_no_more = False
                    new_tup_L = (mult_L, level)
                    new_tup_K = (mult_L, level)
                    #print('new_tup_L:', new_tup_L)
                    mult_to_point_L[new_tup_L] = Tl
                    mult_to_point_K[new_tup_K] = Tk
                    for i in range(len(Tl)):
                        point_to_mult_L[Tl[i]] = new_tup_L
                        point_to_mult_K[Tk[i]] = new_tup_K
                    repeated = len(Tl)
                    if repeated not in repeated_mult_L:
                        repeated_mult_L[repeated] = [Tl]
                    else:
                        repeated_mult_L[repeated] += [Tl]
                    if repeated not in a_repeated_mult:
                        a_repeated_mult[repeated] = [Tl]
                    else:
                        a_repeated_mult[repeated] += [Tl]
                    # right now rechecking independence
                    source = []
                    corresponding = []
                    for r in sorted(repeated_mult_L.keys()):
                        for point_lst in repeated_mult_L[r]:
                            for point in point_lst:
                                if isLinearlyIndependent(source + [point]):
                                    source.append(point)
                                    mult = point_to_mult_L[point]
                                    # if another point with this multiplier and level pair is in S
                                    # then the multiplier level pair will be the last element of corresponding
                                    if len(corresponding) != 0:
                                        if corresponding[-1][0] == mult:
                                            corresponding[-1][1] += 1
                                        else:
                                            corresponding.append([mult, 1])
                                    else:
                                        corresponding.append([mult, 1])
                                if len(source) == n+2:
                                    more = False
                                    break
                            if len(source) == n+2:
                                break
                        if len(source) == n+2:
                            break
                    print(source)
                if not more:
                    break
            if not more:
                break
        print(point_to_mult_L)
        if found_no_more:
            all_points = []
            for r in sorted(repeated_mult_L.keys()):
                for point_lst in repeated_mult_L[r]:
                    all_points += [(pnt, r) for pnt in point_lst]
            best_source = []
            lowest_iterations = -1
            for subset in Subsets(range(len(all_points)), n+2):
                source = []
                for i in subset:
                    source.append(all_points[i][0])
                if isLinearlyIndependent(source):
                    corresponding = []
                    mult_only = []
                    for i in subset:
                        mult = point_to_mult_L[all_points[i][0]]
                        if mult in mult_only:
                            corresponding[mult_only.index(mult)][1] += 1
                        else:
                            corresponding.append([mult, 1])
                            mult_only.append(mult)
                    more = False
                    print(source)
            if more:
                raise ValueError('no more rational preimages')
        i_repeated_mult = deepcopy(a_repeated_mult)
        a_repeated_mult = {}
        found_no_more = True
        #print('i_repeated_mult:', i_repeated_mult)
    #print('source:', source)
    #print('mult_to_point_L:', mult_to_point_L)
    #print('mult_to_point_K:', mult_to_point_K)
    #print('point_to_mult_L:', point_to_mult_L)
    #print('point_to_mult_K:', point_to_mult_K)
    #print('repeated_mult_L keys:', sorted(repeated_mult_L.keys()))
    possible_targets = []
    subset_iterators = []
    for tup in corresponding:
        possible_targets.append([mult_to_point_K[tup[0]], tup[1]])
    #print('possible_targets:', possible_targets)
    for lst in possible_targets:
        subset_iterators.append(Subsets(range(len(lst[0])), lst[1]))
    #print('subset_iterators:', subset_iterators)
    #print('cardinalities:', [itr.cardinality() for itr in subset_iterators])
    Conj = []

    def find_conjugations_subset(tuples):
        conj = []
        for tup in tuples:
            target_set = []
            for i in range(len(tup)):
                for j in tup[i]:
                    target_set.append(possible_targets[i][0][j])
            if isLinearlyIndependent(target_set):
                subset_arrangements = []
                for subset in tup:
                    subset_arrangements.append(Arrangements(subset, len(subset)))
                for tup in product(*subset_arrangements):
                    current_target = []
                    for i in range(len(tup)):
                        for j in tup[i]:
                            current_target.append(possible_targets[i][0][j])
                    phi = f.domain().point_transformation_matrix(current_target, source)
                    if f.conjugate(phi) == g:
                        conj.append(phi)
        return conj
    
    def find_conjugations_arrangement(tuples):
        conj = []
        for tup in tuples:
            current_target = []
            for i in range(len(tup)):
                for j in tup[i]:
                    current_target.append(possible_targets[i][0][j])
            phi = f.domain().point_transformation_matrix(current_target, source)
            if f.conjugate(phi) == g:
                conj.append(phi)
        return conj

    if num_cpus > 1:
        all_subsets = list(product(*subset_iterators))
        print('length:', len(all_subsets))
        parallel_data = []
        if len(all_subsets) > num_cpus:
            for i in range(num_cpus):
                start = floor(len(all_subsets)*i/num_cpus)
                end = floor(len(all_subsets)*(i+1)/num_cpus)
                tuples = all_subsets[start:end]
                parallel_data.append(([tuples], {}))

            X = p_iter_fork(num_cpus)
            for ret in X(find_conjugations_subset, parallel_data):
                if ret[1]:
                    Conj += ret[1]
        else:
            good_targets = []
            for tup in product(*subset_iterators):
                target_set = []
                for i in range(len(tup)):
                    for j in tup[i]:
                        target_set.append(possible_targets[i][0][j])
                if isLinearlyIndependent(target_set):
                    good_targets.append(tup)
            all_arrangements = []
            for tup in good_targets:
                subset_arrangements = []
                for subset in tup:
                    subset_arrangements.append(Arrangements(subset, len(subset)))
                all_arrangements += list(product(*subset_arrangements))
            parallel_data = []
            for i in range(num_cpus):
                start = floor(len(all_arrangements)*i/num_cpus)
                end = floor(len(all_arrangements)*(i+1)/num_cpus)
                tuples = all_arrangements[start:end]
                parallel_data.append(([tuples], {}))
            X = p_iter_fork(num_cpus)
            for ret in X(find_conjugations_arrangement, parallel_data):
                if ret[1]:
                    Conj += ret[1]
    else:
        Conj = find_conjugations_subset(product(*subset_iterators))
    return Conj

In [11]:
%%timeit
P.<x,y,z> = ProjectiveSpace(QQ, 2)
f = DynamicalSystem([x^3, y^3, z^3])
conj_set_preimage(f, f)

67.5 ms ± 913 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:
%%time
K.<v>=CyclotomicField(3)
P.<x,y,z> = ProjectiveSpace(K, 2)
f = DynamicalSystem([8*x^7 - 35*x^4*y^3 - 35*x^4*z^3 - 7*x*y^6 - 140*x*y^3*z^3 - 7*x*z^6, -7*x^6*y - 35*x^3*y^4 - 140*x^3*y*z^3 + 8*y^7 - 35*y^4*z^3 - 7*y*z^6,-7*x^6*z - 140*x^3*y^3*z - 35*x^3*z^4 - 7*y^6*z - 35*y^3*z^4 + 8*z^7])
auto = conj_set_preimage(f, f, 4)

length: 126
CPU times: user 1.32 s, sys: 82.9 ms, total: 1.4 s
Wall time: 5.96 s


In [30]:
%%timeit
P.<x,y,z,w,j,t,h> = ProjectiveSpace(QQ, 6)
f = DynamicalSystem([x^2, y^2, z^2, w^2, j^2, t^2, h^2])
auto = conj_set_preimage(f, f, 4)

6.7 s ± 280 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
%%time
P.<x,y,z,w,t,h> = ProjectiveSpace(QQ, 5)
f = DynamicalSystem([x^3, y^3, z^3, w^3, t^3, h^3])
auto = conj_set_preimage(f, f, 14)

length: 32
CPU times: user 4.62 s, sys: 31.5 ms, total: 4.65 s
Wall time: 15.1 s


In [61]:
len(auto)

23040

In [18]:
K.<v>=CyclotomicField(3)
P.<x,y,z> = ProjectiveSpace(QQ, 2)
f = DynamicalSystem([8*x^7 - 35*x^4*y^3 - 35*x^4*z^3 - 7*x*y^6 - 140*x*y^3*z^3 - 7*x*z^6, -7*x^6*y - 35*x^3*y^4 - 140*x^3*y*z^3 + 8*y^7 - 35*y^4*z^3 - 7*y*z^6,-7*x^6*z - 140*x^3*y^3*z - 35*x^3*z^4 - 7*y^6*z - 35*y^3*z^4 + 8*z^7])
%prun auto = conj_set_preimage(f, f)

In [21]:
P.<x,y> = ProjectiveSpace(QQ, 1)
f = DynamicalSystem_projective([3*x^2*y - y^3, x^3 - 3*x*y^2])
conj_set_preimage(f, f)

ValueError: no more rational preimages

In [3]:
from sage.parallel.use_fork import p_iter_fork
X = p_iter_fork(2, verbose=True)
list(X( (lambda x: x^2), [([10],{}), ([20],{})]))


9253
9254


[(([10], {}), 100), (([20], {}), 400)]

In [16]:
%%time
P.<x,y,z> = ProjectiveSpace(GF(2), 2)
f = DynamicalSystem([x^2, y^2, z^2])
k = conj_set_preimage(f, f, 2)

length: 35
13204
13205
CPU times: user 42.1 ms, sys: 13.4 ms, total: 55.6 ms
Wall time: 154 ms


In [9]:
sage: P.<x,y,z> = ProjectiveSpace(QQ,2)
            sage: f = DynamicalSystem_projective([2*x^2 + 12*y*x, 11*y*x+2*y^2, x^2+z^2])
            sage: m1 = matrix(QQ, 3, 3, [1,4,1,0,2,1,1,1,1])
            sage: g = f.conjugate(m1)
            sage: conj_set_preimage(f, g)

[(0 : 1/2 : 1), (0 : 0 : 1), (1 : 0 : 1)]
[(0 : 1/2 : 1), (0 : 0 : 1), (1 : 0 : 1)]
{(0 : 1/2 : 1): (x^2 - 8*x + 12, 0), (0 : 0 : 1): (x^2, 0), (0 : 1 : 0): (x^2 - 6*x, 0), (1 : 0 : 1): (x^2 - 13/2*x + 11/2, 0), (0 : -1/2 : 1): (x^2 - 8*x + 12, 1), (-1 : 0 : 1): (x^2 - 13/2*x + 11/2, 1)}
{(0 : 1/2 : 1): (x^2 - 8*x + 12, 0), (0 : 0 : 1): (x^2, 0), (0 : 1 : 0): (x^2 - 6*x, 0), (1 : 0 : 1): (x^2 - 13/2*x + 11/2, 0), (0 : -1/2 : 1): (x^2 - 8*x + 12, 1), (-1 : 0 : 1): (x^2 - 13/2*x + 11/2, 1)}
[(0 : 1/2 : 1), (0 : 1 : 0), (1 : 0 : 1), (-1 : 0 : 1)]
[(0 : 1/2 : 1), (1 : 0 : 1), (0 : -1/2 : 1), (-1 : 0 : 1)]
[(0 : 1 : 0), (1 : 0 : 1), (0 : -1/2 : 1), (-1 : 0 : 1)]


[
[1 4 1]
[0 2 1]
[1 1 1]
]

In [13]:
%%timeit
sage: P.<x,y,z> = ProjectiveSpace(QQ, 2)
            sage: f = DynamicalSystem_projective([2*x^2 + 12*y*x, 11*y*x+2*y^2, x^2+z^2])
            sage: m1 = matrix(QQ, 3, 3, [1,4,1,0,2,1,1,1,1])
            sage: g = f.conjugate(m1)
            sage: f.is_conjugate(g, num_cpus=2)

114 ms ± 662 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
